# Inverted pendulu - custom neural network

Install dependencies

In [3]:
!pip3 install tensorflow
!pip3 install gym
!pip3 install keras
!pip3 install keras-rl2

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [4]:
import gym
import random

In [5]:
env = gym.make("CartPole-v0")
states = env.observation_space.shape[0] # get number of observation space variables
actions = env.action_space.n # get number of action space variables

In [6]:
actions

2

In [7]:
episodes = 10

for episode in range(1, episodes + 1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode {} Score {}'.format(episode, score))

env.close()

Episode 1 Score 36.0
Episode 2 Score 24.0
Episode 3 Score 9.0
Episode 4 Score 18.0
Episode 5 Score 34.0
Episode 6 Score 12.0
Episode 7 Score 10.0
Episode 8 Score 31.0
Episode 9 Score 21.0
Episode 10 Score 21.0


Create a Deeo Learning Model with Keras

In [16]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers.legacy import Adam

In [9]:
"""
states -> numbers of variables given as a output (input)
actions -> numebr of actions that neural network can do (output)
"""
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1, states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model


In [10]:
model = build_model(states, actions)

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 4)                 0         
                                                                 
 dense (Dense)               (None, 24)                120       
                                                                 
 dense_1 (Dense)             (None, 24)                600       
                                                                 
 dense_2 (Dense)             (None, 2)                 50        
                                                                 
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


Build agent with Keras-RL

In [12]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy  # policy based rf
from rl.memory import SequentialMemory

In [13]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=5000, window_length = 1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)

    return dqn

In [14]:
model = build_model(states, actions)

In [17]:
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=5000, visualize=False, verbose=1)

2023-04-30 22:06:37.557132: W tensorflow/c/c_api.cc:300] Operation '{name:'dense_5_2/kernel/Assign' id:412 op device:{requested: '', assigned: ''} def:{{{node dense_5_2/kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense_5_2/kernel, dense_5_2/kernel/Initializer/stateless_random_uniform)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


Training for 5000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 8:09 - reward: 1.0000

/home/kuba/.local/lib/python3.10/site-packages/keras/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2023-04-30 22:06:37.875126: W tensorflow/c/c_api.cc:300] Operation '{name:'dense_5/BiasAdd' id:149 op device:{requested: '', assigned: ''} def:{{{node dense_5/BiasAdd}} = BiasAdd[T=DT_FLOAT, _has_manual_control_dependencies=true, data_format="NHWC"](dense_5/MatMul, dense_5/BiasAdd/ReadVariableOp)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-04-30 22:06:37.892492: W tensorflow/c/c_api.cc:300] Operation '{name:'total_3/Assign' id:575 op device:{requested: '', assigned: ''} def:{{{node total_3/Assign}} = AssignVariableOp[_has_manual_control_dependen

   12/10000 [..............................] - ETA: 5:07 - reward: 1.0000

2023-04-30 22:06:38.072754: W tensorflow/c/c_api.cc:300] Operation '{name:'loss_5/AddN' id:676 op device:{requested: '', assigned: ''} def:{{{node loss_5/AddN}} = AddN[N=2, T=DT_FLOAT, _has_manual_control_dependencies=true](loss_5/mul, loss_5/mul_1)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-04-30 22:06:38.097537: W tensorflow/c/c_api.cc:300] Operation '{name:'training/Adam/dense_5/kernel/v/Assign' id:888 op device:{requested: '', assigned: ''} def:{{{node training/Adam/dense_5/kernel/v/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](training/Adam/dense_5/kernel/v, training/Adam/dense_5/kernel/v/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Eit

 4998/10000 [=============>................] - ETA: 37s - reward: 1.0000done, took 37.316 seconds


In [20]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 155.000, steps: 155
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 173.000, steps: 173
Episode 5: reward: 155.000, steps: 155
Episode 6: reward: 191.000, steps: 191
Episode 7: reward: 170.000, steps: 170
Episode 8: reward: 161.000, steps: 161
Episode 9: reward: 167.000, steps: 167
Episode 10: reward: 163.000, steps: 163
Episode 11: reward: 141.000, steps: 141
Episode 12: reward: 199.000, steps: 199
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 173.000, steps: 173
Episode 15: reward: 189.000, steps: 189
Episode 16: reward: 197.000, steps: 197
Episode 17: reward: 163.000, steps: 163
Episode 18: reward: 151.000, steps: 151
Episode 19: reward: 164.000, steps: 164
Episode 20: reward: 179.000, steps: 179
Episode 21: reward: 189.000, steps: 189
Episode 22: reward: 165.000, steps: 165
Episode 23: reward: 185.000, steps: 185
Episode 24: reward: 158.000, steps: 158
Episode 25: reward: 

Reload agent from memory

In [ ]:
_ = dqn.test(env, nb_episodes=100, visualize=True)